In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# Acquire Data


In [62]:
# bring in sample trade log from site
trades = pd.read_csv('trades_sample.csv')

In [63]:
# view df and all columns
pd.set_option('display.max_columns', None)
trades 

,Unnamed: 0,account_type,action,activity_code,amount,as_of_date,check_number,commission,commission_in_gross,contra_broker,cusip,description,effective_date,entry_date,exchange_rate,fees,from_location,front_exec,held_currency_code,interest,maturity_date,misc_fee,nfa_fee,order_number,output_currency_code,payee,price,principal,quantity,sales_credit,sec_fee,security_class_code,security_type,state_tax,symbol,tax_exempt_status,taxes,to_location,trade,transaction_date,user_defined_action
0,0,INVENTORY,Sale,NaN,158632.00,NaN,NaN,0,Y,NaN,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,"Thu, 03 Sep 2020 12:00:00 am EDT","Tue, 01 Sep 2020 2:23:10 am EDT",1.0,0,NaN,NaN,USD,1525.00,NaN,0,0,N0001F99,USD,NaN,104.738,-157107.00,-150000,0,0,NaN,FIXED,0,FIXED 01639PBJ9,NaN,0,NaN,Y,"Tue, 01 Sep 2020 12:00:00 am EDT",NaN
1,1,INVENTORY,Purchase,NaN,-84760.04,NaN,NaN,0,N,NaN,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,"Thu, 03 Sep 2020 12:00:00 am EDT","Tue, 01 Sep 2020 4:36:57 am EDT",1.0,0,NaN,VSIN,USD,19.44,NaN,0,0,N0007F99,USD,NaN,121.058,84740.60,70000,0,0,NaN,FIXED,0,FIXED 64580CFQ5,NaN,0,NaN,Y,"Tue, 01 Sep 2020 12:00:00 am EDT",NaN
2,2,INVENTORY,Sale,NaN,85589.47,NaN,NaN,0,N,NaN,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,"Fri, 04 Sep 2020 12:00:00 am EDT","Wed, 02 Sep 2020 9:10:24 am EDT",1.0,0,NaN,NaN,USD,29.17,NaN,0,0,N0201F99,USD,NaN,122.229,-85560.30,-70000,0,0,NaN,FIXED,0,FIXED 64580CFQ5,NaN,0,NaN,Y,"Wed, 02 Sep 2020 12:00:00 am EDT",NaN
3,3,INVENTORY,Sale,NaN,125835.30,NaN,NaN,0,Y,NaN,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,"Thu, 10 Sep 2020 12:00:00 am EDT","Wed, 02 Sep 2020 1:19:33 am EDT",1.0,0,NaN,NaN,USD,0.00,NaN,0,0,N0401F99,USD,NaN,109.422,-125835.30,-115000,0,0,NaN,FIXED,0,FIXED 054231QS7,NaN,0,NaN,Y,"Wed, 02 Sep 2020 12:00:00 am EDT",NaN
4,4,INVENTORY,Purchase,NaN,-367239.00,NaN,NaN,0,Y,NaN,355525FA9,FRASER VY MET RECREATIONAL DIS REF,"Wed, 16 Sep 2020 12:00:00 am EDT","Wed, 02 Sep 2020 10:21:47 am EDT",1.0,0,NaN,NaN,USD,0.00,NaN,0,0,N0303F99,USD,NaN,122.413,367239.00,300000,0,0,NaN,FIXED,0,FIXED 355525FA9,NaN,0,NaN,Y,"Wed, 02 Sep 2020 12:00:00 am EDT",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,INVENTORY,Sale,NaN,68507.60,NaN,NaN,0,Y,NaN,368414AV0,GEAUGA CNTY OHIO SPL OBLIG SAL BDS ...,"Tue, 20 Jul 2021 12:00:00 am EDT","Fri, 16 Jul 2021 4:58:20 am EDT",1.0,0,NaN,NaN,USD,315.00,NaN,0,0,N0001F99,USD,NaN,97.418,-68192.60,-70000,0,0,NaN,FIXED,0,FIXED 368414AV0,NaN,0,NaN,Y,"Fri, 16 Jul 2021 12:00:00 am EDT",NaN
351,351,INVENTORY,Sale,NaN,73839.69,NaN,NaN,0,N,NaN,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,"Tue, 20 Jul 2021 12:00:00 am EDT","Fri, 16 Jul 2021 9:11:58 am EDT",1.0,0,NaN,VSIN,USD,185.94,NaN,0,0,N0003F99,USD,NaN,98.205,-73653.75,-75000,0,0,NaN,FIXED,0,FIXED 800851TV9,NaN,0,NaN,Y,"Fri, 16 Jul 2021 12:00:00 am EDT",NaN
352,352,INVENTORY,Sale,NaN,73873.44,NaN,NaN,0,N,NaN,800851TV9,SANGER CALIF UNI SCH DIST ELECTION OF 2018 GO ...,"Tue, 20 Jul 2021 12:00:00 am EDT","Fri, 16 Jul 2021 1:06:58 pm EDT",1.0,0,NaN,NaN,USD,185.94,NaN,0,0,N0004F99,USD,NaN,98.250,-73687.50,-75000,0,0,NaN,FIXED,0,FIXED 800851TV9,NaN,0,NaN,Y,"Fri, 16 Jul 2021 12:00:00 am EDT",NaN
353,353,INVENTORY,Sale,NaN,111805.67,NaN,NaN,0,N,NaN,018112SY9,ALLEN TEX WTRWKS & SWR REV SYS BDS ...,"Thu, 29 Jul 2021 12:00:00 am EDT","Tue, 27 Jul 2021 7:40:51 am EDT",1.0,0,NaN,NaN,USD,241.67,NaN,0,0,N0001F99,USD,NaN,111.564,-111564.00,-100000,0,0,NaN,FIXED,0,FIXED 018112SY9,NaN,0,NaN,Y,"Tue, 27 Jul 2021 12:00:00 am EDT",NaN


# Clean Data

In [64]:
# Drop unnecessary columns
trades = trades.drop(columns=['Unnamed: 0', 'amount', 'account_type', 'activity_code', 'as_of_date', 'check_number', 'commission', 'commission_in_gross', 'contra_broker', 'exchange_rate', 'fees', 'from_location', 'front_exec', 'held_currency_code', 'interest', 'maturity_date', 'misc_fee', 'nfa_fee', 'order_number', 'output_currency_code', 'payee', 'principal', 'sales_credit', 'sec_fee', 'security_class_code', 'security_type', 'state_tax', 'symbol', 'tax_exempt_status', 'taxes', 'to_location', 'trade',  'user_defined_action'])

In [66]:
# Divide trades by 1000 per bond quantity
trades['quantity'] = (trades.quantity)/1000

In [67]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   action            355 non-null    object 
 1   cusip             355 non-null    object 
 2   description       355 non-null    object 
 3   effective_date    355 non-null    object 
 4   entry_date        355 non-null    object 
 5   price             355 non-null    float64
 6   quantity          355 non-null    float64
 7   transaction_date  355 non-null    object 
dtypes: float64(2), object(6)
memory usage: 22.3+ KB


In [68]:
def convert_date(feature):
    '''takes date/time object, drops timestamp, converts to datetime mm/dd/yyy format'''
    # use regex to drop leading 5 characters (e.g. Thu, )
    trades[feature] = trades[feature].str.replace(r'^(.{5})','')
    # use regex to drop timestamp section at end of string
    trades[feature] = trades[feature].str.replace(r'(.{15})\.*$','')
    # convert object to datetime format
    trades[feature] = pd.to_datetime(trades[feature])
    # format to mm/dd/yyyy format
    trades[feature] = trades[feature].apply(lambda x: x.strftime("%m/%d/%Y"))
    return trades[feature]

In [69]:
trades['transaction_date'] = convert_date('transaction_date')
trades['entry_date'] = convert_date('entry_date')
trades['effective_date'] = convert_date('effective_date')

<ipython-input-68-7ba02226cc12>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  trades[feature] = trades[feature].str.replace(r'^(.{5})','')
<ipython-input-68-7ba02226cc12>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  trades[feature] = trades[feature].str.replace(r'(.{15})\.*$','')


In [70]:
trades.head()

,action,cusip,description,effective_date,entry_date,price,quantity,transaction_date
0,Sale,01639PBJ9,ALISO VIEJO CALIF CTFS PARTN COPS ...,09/03/2020,09/01/2020,104.738,-150.0,09/01/2020
1,Purchase,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,09/03/2020,09/01/2020,121.058,70.0,09/01/2020
2,Sale,64580CFQ5,NEW JERSEY INFRASTRUCTURE BK ENVIRONMENT REF B...,09/04/2020,09/02/2020,122.229,-70.0,09/02/2020
3,Sale,054231QS7,AVON OHIO LOC SCH DIST GO UNLTD TAX IMPT BDS ...,09/10/2020,09/02/2020,109.422,-115.0,09/02/2020
4,Purchase,355525FA9,FRASER VY MET RECREATIONAL DIS REF,09/16/2020,09/02/2020,122.413,300.0,09/02/2020
